In [137]:
import numpy as np
import matplotlib.pyplot as plt
import pandas

In [138]:
data = np.load('notMNIST.npz')
print(data.files)

['images', 'labels']


In [139]:
with np.load('notMNIST.npz') as data :
    Data, Target = data ['images'], data['labels']
    posClass = 2
    negClass = 9
    dataIndx = (Target==posClass) + (Target==negClass)
    Data = Data[dataIndx]/255.
    Target = Target[dataIndx].reshape(-1, 1)
    Target[Target==posClass] = 1
    Target[Target==negClass] = 0
    np.random.seed(421)
    randIndx = np.arange(len(Data))
    np.random.shuffle(randIndx)
    Data, Target = Data[randIndx], Target[randIndx]
    trainData, trainTarget = Data[:3500], Target[:3500]
    validData, validTarget = Data[3500:3600], Target[3500:3600]
    testData, testTarget = Data[3600:], Target[3600:]  
   

In [140]:
#Reshaping traiing,testing and validating data
train_X = trainData.reshape(3500,784)
X0 = np.ones((3500,1))
train_X = np.append(X0,train_X,axis=1)
train_Y = trainTarget

test_X = testData.reshape(145,784)
X0 = np.ones((145,1))
test_X = np.append(X0,test_X,axis=1) 
test_Y = testTarget

validation_X = validData.reshape(100,784)
X0 = np.ones((100,1))
validation_X = np.append(X0,validation_X,axis=1)
validation_Y = validTarget





In [154]:
def sigmoid(z):
    s = 1.0/(1.0 + np.exp(-1.0 * z))    
    return s

In [155]:
def crossEntropyLoss(W, b, x, y, reg):
    #print('W:', W.shape)
    #print('b:',b.shape)       
    w_new = np.concatenate((b,W))    
    loss_d = (-y * np.log(x) - (1 - y) * np.log(1 - x)).mean()              
    loss_w = (reg/2) * (np.dot(w_new.T,w_new))  
    tot_loss = loss_d + loss_w 
    #print(loss_d)
    #print(loss_w)    
    return tot_loss     
    

In [156]:
def propagate(w, b, X_train, Y_train,reg):
    
    m = X_train.shape[0]
    w = np.concatenate((b,w))
    Z= np.matmul(np.transpose(w), X_train.T)
    A = sigmoid(Z)
    cost = (-1.0) * np.mean(np.multiply(Y_train, np.log(A)) + np.multiply(1.0-Y_train, np.log(1.0 - A)), axis=1)                              # compute cost
    
    dw = np.matmul(np.transpose(X_train), np.transpose(A - Y_train)) * (1.0/m)
    db = np.mean((A - Y_train))
    
    cost = np.squeeze(cost)
    
    grads = {"dw": dw[1:],
             "db": db}
    
    return grads, cost

In [157]:
def grad_descent(w, b, X_train, Y_train, epochs, alpha, reg, EPS):
    
    loss_train =[]
    loss_validation = []
    iteration = []
    loss_test = []
    
    theta = np.zeros((785,1))
    theta1 = np.zeros((785,1))
    
    for i in range(epochs):
        
        #Initializing weight and bias
        w = np.random.rand(784,1)
        b = np.random.rand(1,1)
        
        z = np.dot(X_train, theta)
        #print(z)
        h = sigmoid(z)
        #print(h)
        gradient = np.dot(X_train.T, (h - Y_train))
        theta = np.transpose(theta)- alpha * gradient
        
        z = np.dot(X_train, theta)
        h = sigmoid(z)
        
        loss = crossEntropyLoss(w,b,h,Y_train,reg)
        loss_train.append(loss)
        iteration.append(i)
        
        # Cost and gradient calculation 
        grads, cost = propagate(w, b, X_train, Y_train, reg)
        
        # Retrieve derivatives from grads
        dw = grads["dw"]
        db = grads["db"]
        
        # update rule
        w = w - alpha * dw
        b = b - alpha * db
        
        if i % 1 == 0 :
            w1 = np.random.rand(784,1)
            b1 = np.random.rand(1,1)
            z1 = np.dot(validation_X, theta1)
            #print(z)
            h1 = sigmoid(z1)
            #print(h)
            gradient1 = np.dot(validation_X.T, (h1 - validation_Y))
            theta1 = np.transpose(theta1) - alpha * gradient1

            z1 = np.dot(validation_X, theta1)
            h1 = sigmoid(z1)
            loss_val = crossEntropyLoss(w1,b1,h1,validation_Y,reg)
            loss_validation.append(loss_val)
            print ("Iterations : " + str(i) +  "  Loss : " + str(loss) +" Validation Loss : " + str(loss_val))
        
        if i is 0 :
            least_loss = loss
            error = 0 
        else :
            error = prev - loss
            
        if loss <= least_loss :
            least_loss = loss 
            W_final = W
            b_final = b
        if error< EPS and i is not 0:
            print(error)
            break        

        prev = loss
        
    loss_test = crossEntropyLoss(w,b,test_X,test_Y,reg)
    #plot_print_loss(iterations, loss_train,loss_validation,loss_test)
    return b , W, loss_train, loss_validation,loss_test, iteration

In [158]:
def model(train_X,train_Y):  
    
 
    #Initializing weight and bias
    w = np.random.rand(784,1)
    b = np.random.rand(1,1)
    # Gradient descent (≈ 1 line of code)
    b, W, loss_train, loss_validation, loss_test, iteration = grad_descent(w, b, train_X, train_Y, 5000, 0.005, reg = 0.0, EPS = 1e-7)
    return loss_train, loss_test, loss_validation 
    

In [159]:
loss_train, loss_test, loss_validation = model(train_X,train_Y)

C:\Users\Urmi Joshi\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: RuntimeWarning: divide by zero encountered in log
  import sys
C:\Users\Urmi Joshi\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: RuntimeWarning: invalid value encountered in multiply
  import sys


Iterations : 0  Loss : [[ nan]] Validation Loss : [[-8.18255291]]


C:\Users\Urmi Joshi\Anaconda3\lib\site-packages\ipykernel_launcher.py:65: RuntimeWarning: invalid value encountered in less_equal
C:\Users\Urmi Joshi\Anaconda3\lib\site-packages\ipykernel_launcher.py:12: RuntimeWarning: overflow encountered in exp
  if sys.path[0] == '':


Iterations : 1  Loss : [[ nan]] Validation Loss : [[ nan]]


C:\Users\Urmi Joshi\Anaconda3\lib\site-packages\ipykernel_launcher.py:69: RuntimeWarning: invalid value encountered in less


Iterations : 2  Loss : [[ nan]] Validation Loss : [[ nan]]
Iterations : 3  Loss : [[ nan]] Validation Loss : [[ nan]]
Iterations : 4  Loss : [[ nan]] Validation Loss : [[ nan]]
Iterations : 5  Loss : [[ nan]] Validation Loss : [[ nan]]
Iterations : 6  Loss : [[ nan]] Validation Loss : [[ nan]]
Iterations : 7  Loss : [[ nan]] Validation Loss : [[ nan]]
Iterations : 8  Loss : [[ nan]] Validation Loss : [[ nan]]
Iterations : 9  Loss : [[ nan]] Validation Loss : [[ nan]]
Iterations : 10  Loss : [[ nan]] Validation Loss : [[ nan]]
Iterations : 11  Loss : [[ nan]] Validation Loss : [[ nan]]
Iterations : 12  Loss : [[ nan]] Validation Loss : [[ nan]]
Iterations : 13  Loss : [[ nan]] Validation Loss : [[ nan]]
Iterations : 14  Loss : [[ nan]] Validation Loss : [[ nan]]
Iterations : 15  Loss : [[ nan]] Validation Loss : [[ nan]]
Iterations : 16  Loss : [[ nan]] Validation Loss : [[ nan]]
Iterations : 17  Loss : [[ nan]] Validation Loss : [[ nan]]
Iterations : 18  Loss : [[ nan]] Validation Loss

KeyboardInterrupt: 

In [ ]:
plt.plot(iterations[200:], loss_train[200:])

plt.plot(iterations[200:], loss_validation[200:])

print("Test loss : "+str(loss_test))
